<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_08_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 08: CatBoost

## Objetivos
- Comprender qué es CatBoost y cómo maneja variables categóricas.
- Entrenar un modelo CatBoost con el dataset "default de tarjetas de crédito" y con el dataset "bank marketing"
- Comparar resultados con XGBoost.

---

## 1. ¿Qué es CatBoost?
- Algoritmo de gradient boosting desarrollado por Yandex.
- **Ventajas clave:**
  - Manejo nativo de **variables categóricas**.
  - Generaliza bien con menos tuning.
  - Menor riesgo de overfitting en comparación con otros boosters.

---

## 2. Preparación del Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
df = pd.read_excel(url, header=1)

df.rename(columns={"default payment next month": "default"}, inplace=True)

In [ ]:
X = df.drop(columns=["ID","default"])
y = df["default"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

---

## 3. Entrenar CatBoost

In [ ]:
# Verificar si catboost está instalado, si no instalarlo
try:
    import catboost
    print("✅ CatBoost ya está instalado")
except ImportError:
    print("⚠️ CatBoost no está instalado. Instalando ahora...")
    !pip install -q catboost > /dev/null
    from catboost import CatBoostClassifier
    print("✅ CatBoost instalado correctamente")


In [ ]:
cat = CatBoostClassifier(
    iterations=200,
    depth=6,
    learning_rate=0.1,
    random_seed=42,
    verbose=50,
    loss_function="Logloss",   # lo que se optimiza
    eval_metric="AUC",         # lo que se usa para seleccionar el "best iteration"
    auto_class_weights="Balanced",
    early_stopping_rounds=100,  # ⚡ se detiene si no mejora en 100 iteraciones
)

cat.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    plot=True
)

# opcional: inspeccionar métricas por iteración
ev = cat.get_evals_result()
print("Best iteration by AUC:", cat.get_best_iteration())
print("Best scores:", cat.get_best_score())  # {'learn': {...}, 'validation': {...}}

y_pred = cat.predict(X_test)
y_proba = cat.predict_proba(X_test)[:,1]

print('\nClassification Report\n',classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))

In [ ]:
# === Matriz de confusión ===
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicción")
plt.ylabel("Valor real")
plt.title("Matriz de confusión")
plt.show()

# === Curva ROC ===
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"XGBClassifier (AUC = {roc_auc:.2f})")
plt.plot([0,1], [0,1], linestyle="--", color="gray")  # línea diagonal
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC")
plt.legend(loc="lower right")
plt.show()

In [ ]:
# ------------------------------
# Importancia de variables (Gain)
# ------------------------------
feat_imp = pd.DataFrame({
    'feature': X_train.columns,
    'importance': cat.get_feature_importance(type='FeatureImportance')
}).sort_values('importance', ascending=False)

# Plot Top-N
TOP_N = 20
plt.figure(figsize=(8, max(4, TOP_N * 0.3)))
sns.barplot(data=feat_imp.head(TOP_N), x='importance', y='feature')
plt.title(f'Importancia de variables (Top {TOP_N}) - CatBoost')
plt.xlabel('Importancia (Gain)')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

In [ ]:
# ------------------------------
# SHAP values (Beeswarm)
# ------------------------------
import shap
# Para velocidad, tomamos una muestra (ajustar el tamaño a gusto)
sample_size = min(5000, len(X_test))
X_sample = X_test.sample(sample_size, random_state=42)

# SHAP con CatBoost: usar predict con pred_type='RawFormulaVal' para background;
# pero shap.TreeExplainer maneja CatBoost directamente.
explainer = shap.TreeExplainer(cat)
shap_values = explainer.shap_values(X_sample)

# Si el modelo es binario, shap_values puede ser un array 2D (n_samples, n_features)
# o una lista de arrays. Normalizamos a una matriz 2D:
if isinstance(shap_values, list):
    # Para binario en versiones recientes, suele devolver 2 clases; tomamos la de la clase positiva
    shap_values = shap_values[1]

shap.summary_plot(shap_values, X_sample, plot_type="dot", show=True)  # beeswarm

---
## Ejemplo adicional: CatBoost con Bank Marketing (UCI)
Bank Marketing (UCI) con desbalance, métricas y explicabilidad

In [ ]:
import io, zipfile, requests, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, roc_auc_score, roc_curve, confusion_matrix
)

from catboost import CatBoostClassifier

In [ ]:
# ------------------------------
# 1) Cargar dataset desde ZIP (UCI)
# ------------------------------
url_bank_zip = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip"
resp = requests.get(url_bank_zip, timeout=60)
resp.raise_for_status()
z = zipfile.ZipFile(io.BytesIO(resp.content))
csv_path_inside_zip = "bank-additional/bank-additional-full.csv"

bank_df = pd.read_csv(z.open(csv_path_inside_zip), sep=';')

In [ ]:
# ------------------------------
# 2) Preparación: target, split y categóricas
# ------------------------------
X_bank = bank_df.drop("y", axis=1)
y_bank = bank_df["y"].map({"yes": 1, "no": 0})

cat_features_bank = X_bank.select_dtypes(include=['object']).columns.tolist()
print("Columnas categóricas:", cat_features_bank)

Xb_train, Xb_test, yb_train, yb_test = train_test_split(
    X_bank, y_bank, test_size=0.3, random_state=42, stratify=y_bank
)

In [ ]:
# ------------------------------
# 3) Manejo de desbalance mediante class_weights
#    (inversamente proporcional a la frecuencia de cada clase)
# ------------------------------
pos_rate = yb_train.mean()
w1 = 0.5 / pos_rate          # peso clase 1
w0 = 0.5 / (1.0 - pos_rate)  # peso clase 0
class_weights = {0: w0, 1: w1}
print(f"Pesos de clase -> 0: {w0:.3f}, 1: {w1:.3f}")

# ------------------------------
# 4) Entrenamiento CatBoost con eval_set y registro de métricas
#    (eval_metric AUC para seguimiento)
# ------------------------------
cat_bank = CatBoostClassifier(
    iterations=600,
    depth=6,
    learning_rate=0.07,
    random_seed=42,
    verbose=50,
    eval_metric="AUC",
    loss_function="Logloss",
    #class_weights=class_weights,
    auto_class_weights="Balanced"  # ✅ ajuste automático del desbalance
)

cat_bank.fit(
    Xb_train, yb_train,
    cat_features=cat_features_bank,
    eval_set=(Xb_test, yb_test),
    use_best_model=True,
    plot=True
)

In [ ]:
# ------------------------------
# 5) Predicciones y métricas básicas
# ------------------------------
yb_pred = cat_bank.predict(Xb_test)
yb_proba = cat_bank.predict_proba(Xb_test)[:, 1]

print("\nReporte de clasificación (Bank Marketing):")
print(classification_report(yb_test, yb_pred, digits=4))
print("AUC:", roc_auc_score(yb_test, yb_proba))

In [ ]:
# ------------------------------
# 6) Curva ROC
# ------------------------------
fpr, tpr, thr = roc_curve(yb_test, yb_proba)
auc_ = roc_auc_score(yb_test, yb_proba)

plt.figure()
plt.plot(fpr, tpr, label=f'ROC (AUC={auc_:.3f})')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Curva ROC - Bank Marketing')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ------------------------------
# 8) Matriz de confusión (normalizada por filas)
# ------------------------------
cm = confusion_matrix(yb_test, yb_pred)

plt.figure()
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['No', 'Sí'], yticklabels=['No', 'Sí'])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de confusión")
plt.show()


In [ ]:
# ------------------------------
# 9) Importancia de variables (Gain)
# ------------------------------
feat_imp = pd.DataFrame({
    'feature': Xb_train.columns,
    'importance': cat_bank.get_feature_importance(type='FeatureImportance')
}).sort_values('importance', ascending=False)

# Plot Top-N
TOP_N = 20
plt.figure(figsize=(8, max(4, TOP_N * 0.3)))
sns.barplot(data=feat_imp.head(TOP_N), x='importance', y='feature')
plt.title(f'Importancia de variables (Top {TOP_N}) - CatBoost')
plt.xlabel('Importancia (Gain)')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

In [ ]:
# ------------------------------
# 10) SHAP values (Beeswarm)
# ------------------------------

from catboost import Pool
import numpy as np
import pandas as pd

# 1) Pool de test con info de categóricas
test_pool = Pool(
    data=Xb_test,
    label=yb_test,
    cat_features=cat_features_bank
)

# 2) SHAP nativo de CatBoost
#    Resultado: (n_muestras, n_features + 1); última col = base value (expected value)
shap_raw = cat_bank.get_feature_importance(
    test_pool,
    type='ShapValues'
)

# 3) Separar contribuciones y valor base
shap_values = shap_raw[:, :-1]   # contribuciones por feature
base_value  = shap_raw[:, -1]    # expected value (no necesario para el beeswarm)

# 4) Opcional: muestrear para velocidad
sample_size = min(5000, len(Xb_test))
X_sample = Xb_test.sample(sample_size, random_state=42)
idx = X_sample.index
shap_values_sample = shap_values[np.isin(Xb_test.index, idx), :]

# Convertir categóricas a string para visualización
X_sample_disp = X_sample.copy()
for c in cat_features_bank:
    X_sample_disp[c] = X_sample_disp[c].astype(str)

# 5) Graficar beeswarm con shap
import shap
shap.summary_plot(
    shap_values_sample,
    X_sample_disp,
    plot_type="dot",    # beeswarm clásico
    show=True
)

In [ ]:
## Force plot individual (CatBoost SHAP + shap)

import numpy as np
import shap

# Elegimos una instancia a explicar:
# 1) por índice directo (ej.: la primera del test)
i = 0

# 2) o la de mayor probabilidad positiva (ejemplo común):
# i = np.argmax(cat_bank.predict_proba(Xb_test)[:, 1])

# Extraer los insumos de esa instancia
x_row = Xb_test.iloc[i]
shap_row = shap_values[i, :]     # contribuciones SHAP de esa fila (sin base value)
expected = base_value[i]         # expected value (base) para esa fila

# shap necesita features en el mismo formato; convertimos categóricas a str para mostrar bonito
x_row_disp = x_row.copy()
for c in cat_features_bank:
    x_row_disp[c] = str(x_row_disp[c])

# Mostrar el force plot en Jupyter
shap.initjs()
shap.force_plot(
    base_value=expected,
    shap_values=shap_row,
    features=x_row_disp,
    matplotlib=True  # en notebooks HTML interactivo; si preferís imagen estática, usa True
)


## 4. Comparación con XGBoost

Aunque **CatBoost** y **XGBoost** son dos de los algoritmos de *gradient boosting* más utilizados, presentan diferencias importantes:

* **Preprocesamiento de variables categóricas**

  * *CatBoost*: soporta variables categóricas de forma nativa. Utiliza técnicas como *ordered boosting* y *target statistics* que reducen leakage y evitan el one-hot encoding masivo.
  * *XGBoost*: en versiones clásicas requería transformar categorías (p. ej., one-hot o label encoding). Sin embargo, **desde la versión 1.3** y posteriores, XGBoost ya soporta **categorías de manera nativa** (`enable_categorical=True` en `DMatrix`/`XGBClassifier`). Esto reduce la brecha que existía antes.

* **Procesamiento de texto**

  * *CatBoost*: ofrece soporte para **features de texto**, aplicando técnicas inspiradas en NLP (por ejemplo, tokenización interna, embeddings simples y combinaciones de palabras). Esto le permite trabajar con columnas textuales sin requerir una vectorización externa compleja (como TF-IDF o Word2Vec).
  * *XGBoost*: no tiene soporte nativo para texto. Normalmente hay que preprocesarlo (TF-IDF, embeddings, etc.) antes de entrenar.

* **Velocidad y escalabilidad**

  * *XGBoost*: suele ser más rápido en datasets **muy grandes** gracias a su motor altamente optimizado en memoria y paralelización.
  * *CatBoost*: en conjuntos de datos medianos-grandes con muchas variables categóricas puede ser más lento, aunque simplifica mucho el pipeline porque evita pasos manuales de codificación.

* **Calidad del modelo**

  * Ambos logran rendimientos similares en muchos benchmarks.
  * *CatBoost* suele destacar en datasets ricos en variables categóricas o textuales.
  * *XGBoost* puede igualar o superar en datasets mayormente numéricos con gran volumen.

* **Interpretabilidad y comunidad**

  * Ambos disponen de herramientas de interpretación como **Feature Importance** y **SHAP values**.
  * *XGBoost* tiene una comunidad más grande y madura.
  * *CatBoost* sigue creciendo rápido, sobre todo en entornos donde el tabular con muchas categorías es dominante (ej. crédito, marketing, retail).

---

## 📊 Tabla comparativa: CatBoost vs XGBoost

| Aspecto               | **CatBoost**                                                                                         | **XGBoost**                                                                                                    |
| --------------------- | ---------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------- |
| **Categorías**        | Soporta **nativamente** variables categóricas (no requiere encoding manual, usa *ordered boosting*). | Desde la versión **1.3+** soporta categorías nativamente (`enable_categorical=True`), antes requería encoding. |
| **Texto**             | Puede procesar **features textuales** de forma nativa (tokenización, combinaciones de palabras).     | No tiene soporte nativo para texto → se necesita preprocesar (TF-IDF, embeddings, etc.).                       |
| **Velocidad**         | Muy eficiente en datasets medianos con muchas categorías. Puede ser algo más lento en big data.      | Generalmente más **rápido y escalable** en datasets muy grandes (optimizaciones en memoria y paralelismo).     |
| **Comunidad**         | Comunidad más pequeña pero en rápido crecimiento.                                                    | Comunidad muy grande, extensa documentación y ecosistema maduro.                                               |
| **Interpretabilidad** | Soporta SHAP nativo (`get_feature_importance(type="ShapValues")`) + plots.                           | Integración fluida con librerías externas como `shap`. Ambas con opciones de importancia de features.          |

---

### 🔎 Comentarios clave

* **CatBoost**: ideal cuando el dataset tiene **muchas categorías o variables textuales**, porque reduce el preprocesamiento al mínimo.
* **XGBoost**: excelente elección en **datasets muy grandes y mayormente numéricos**, gracias a su velocidad y comunidad extensa.
* Ambos superan a modelos lineales en tabulares con relaciones no lineales, y ofrecen herramientas modernas de interpretabilidad (SHAP, feature importance).

---


## 5. Preguntas de Discusión

1. **¿En qué escenarios CatBoost puede ser preferible sobre XGBoost?**

   * Cuando el dataset tiene **muchas variables categóricas** con alta cardinalidad.
   * Cuando se trabaja con **texto en columnas tabulares** (ejemplo: reseñas de clientes, descripciones de productos).
   * En pipelines donde se busca **menos preprocesamiento manual** y un flujo más sencillo.

2. **¿Por qué CatBoost maneja mejor las variables categóricas?**

   * Porque implementa *ordered boosting* y codificaciones basadas en estadísticas que reducen el leakage.
   * Porque convierte automáticamente las categorías en representaciones numéricas útiles sin inflar la dimensionalidad (a diferencia del one-hot encoding).
   * Además, extiende esta filosofía a **features textuales**, lo que lo hace más versátil que otros boosters.

3. **¿Qué trade-off hay entre interpretabilidad y performance en estos modelos?**

   * Ambos ofrecen **alto performance**, a costa de ser menos interpretables que modelos lineales o árboles simples.
   * Herramientas como **SHAP**, **Feature Importance** o **Partial Dependence Plots** ayudan, pero la explicación sigue siendo más compleja que con modelos más simples.
   * En dominios regulados (ej. salud, finanzas) este trade-off es clave: se busca un balance entre **predicciones más precisas** y **modelos más comprensibles** para auditores o usuarios finales.

---